In [69]:
import pandas as pd
import numpy as np
import json
import os

filepath = os.path.join("Historic_Request_MCO_20190122_9.json")
with open(filepath) as jsonfile:
    mco20190122_json = json.load(jsonfile)
    
    

In [81]:
#isolate data items
# Date Range A: Normal Operating Period 22nd Dec 2017 - 25th Jan 2018
# Date Range B: Shutdown Operating Period 22nd Dec 2018 - 25th Jan 2019
# Time Block: 2pm - 8pm
# Carrier: Delta Airlines (DL)
# Airports: MCO, ATL

# Data Fields: airport code, date, carrier code, 
# Filter by departureGateDelayMinutes, actualGateDeparture

data = mco20190122_json["request"]
flightstatuses = mco20190122_json["flightStatuses"]

MCO_20190122_flights_df = pd.DataFrame(index=np.arange(0, 0), columns=('Counter', 'ID', 'Carrier', 'Number', 'Airport', 'Date', 'Status', 'Scheduled Hour', 'Scheduled Mins', 'Actual Hour', 'Actual Mins', 'Delay') )

Flight_Counter = 0
Flights = []

for getFlight in mco20190122_json['flightStatuses']:
    Flight_Counter = Flight_Counter + 1
    print(f"Flight Counter: {Flight_Counter}")
    myFlightID = getFlight['flightId']
    print(f"FlightID: {myFlightID}")
    myFlightCarrier = getFlight['carrierFsCode']
    print(f"Carrier: {myFlightCarrier}")
    myFlightNum = getFlight['flightNumber']
    print(f"Flight#: {myFlightNum}")
    myFlightDepAirport = getFlight['departureAirportFsCode']
    print(f"Airport: {myFlightDepAirport}")
    myFlightDepDateTime = getFlight['departureDate']['dateLocal']
    myFlightDepDate = myFlightDepDateTime.split("T")[0]
    print(f"Date: {myFlightDepDate}")
    myFlightStatus = getFlight['status']
    print(f"Status: {myFlightStatus}")
    
    try:
        myFlightScheduledDep = getFlight['operationalTimes']['scheduledGateDeparture']['dateLocal']
        myFlightScheduledDepTime = myFlightScheduledDep.split("T")[1]
        myFlightShedDepHour = myFlightScheduledDepTime.split(":")[0]
        myFlightShedDepMinutes = myFlightScheduledDepTime.split(":")[1]
        print(f"Scheduled Dep Time: {myFlightScheduledDepTime} Hour: {myFlightShedDepHour} Minutes: {myFlightShedDepMinutes}")
    
        myFlightActualDep = getFlight['operationalTimes']['actualGateDeparture']['dateLocal']
        myFlightActualDepTime = myFlightActualDep.split("T")[1]
        myFlightActualDepHour = myFlightActualDepTime.split(":")[0]
        myFlightActualDepMinutes = myFlightActualDepTime.split(":")[1]
        print(f"Actual Dep Time: {myFlightActualDepTime} Hour: {myFlightActualDepHour} Minutes: {myFlightActualDepMinutes}")
    
        myFlightDepStartTime = (int(myFlightShedDepHour) * 60) + int(myFlightShedDepMinutes)
        myFlightDepEndTime = (int(myFlightActualDepHour) * 60) + int(myFlightActualDepMinutes)
        myFlightDelayMinutes = myFlightDepEndTime - myFlightDepStartTime
        #print(f"Start Time: {myFlightDepStartTime}")
        #print(f"End Time: {myFlightDepEndTime}")
        print(f"Delay (Minutes): {myFlightDelayMinutes}")
    except:
        print(f"No Gate Depature Delay Data")
    print("\n")
    MCO_20190122_flights_df.loc[Flight_Counter] = [Flight_Counter, myFlightID, myFlightCarrier, myFlightNum, myFlightDepAirport, myFlightDepDate, myFlightStatus, myFlightShedDepHour, myFlightShedDepMinutes, myFlightActualDepHour, myFlightActualDepMinutes, myFlightDelayMinutes]


Flight Counter: 1
FlightID: 987468344
Carrier: NK
Flight#: 432
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 07:35:00.000 Hour: 07 Minutes: 35
Actual Dep Time: 12:23:00.000 Hour: 12 Minutes: 23
Delay (Minutes): 288


Flight Counter: 2
FlightID: 987425422
Carrier: B6
Flight#: 52
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 08:00:00.000 Hour: 08 Minutes: 00
Actual Dep Time: 09:32:00.000 Hour: 09 Minutes: 32
Delay (Minutes): 92


Flight Counter: 3
FlightID: 987442628
Carrier: F9
Flight#: 1011
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 08:20:00.000 Hour: 08 Minutes: 20
Actual Dep Time: 08:47:00.000 Hour: 08 Minutes: 47
Delay (Minutes): 27


Flight Counter: 4
FlightID: 987495301
Carrier: WN
Flight#: 1145
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 08:40:00.000 Hour: 08 Minutes: 40
Actual Dep Time: 09:32:00.000 Hour: 09 Minutes: 32
Delay (Minutes): 52


Flight Counter: 5
FlightID: 987410386
Carrier: AA
Flight#: 1110
Airport: MCO
D

Flight Counter: 74
FlightID: 987498965
Carrier: WS
Flight#: 1347
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 11:25:00.000 Hour: 11 Minutes: 25
Actual Dep Time: 11:25:00.000 Hour: 11 Minutes: 25
Delay (Minutes): 0


Flight Counter: 75
FlightID: 987468606
Carrier: NK
Flight#: 923
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 11:26:00.000 Hour: 11 Minutes: 26
Actual Dep Time: 11:25:00.000 Hour: 11 Minutes: 25
Delay (Minutes): -1


Flight Counter: 76
FlightID: 987495376
Carrier: WN
Flight#: 1210
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 11:30:00.000 Hour: 11 Minutes: 30
Actual Dep Time: 11:25:00.000 Hour: 11 Minutes: 25
Delay (Minutes): -5


Flight Counter: 77
FlightID: 987424811
Carrier: B6
Flight#: 120
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 11:30:00.000 Hour: 11 Minutes: 30
Actual Dep Time: 12:21:00.000 Hour: 12 Minutes: 21
Delay (Minutes): 51


Flight Counter: 78
FlightID: 987434803
Carrier: DL
Flight#: 1723
Airport: M

Status: L
Scheduled Dep Time: 13:46:00.000 Hour: 13 Minutes: 46
Actual Dep Time: 14:23:00.000 Hour: 14 Minutes: 23
Delay (Minutes): 37


Flight Counter: 137
FlightID: 987497949
Carrier: WN
Flight#: 497
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 13:50:00.000 Hour: 13 Minutes: 50
Actual Dep Time: 14:28:00.000 Hour: 14 Minutes: 28
Delay (Minutes): 38


Flight Counter: 138
FlightID: 987489226
Carrier: UA
Flight#: 558
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 13:50:00.000 Hour: 13 Minutes: 50
Actual Dep Time: 13:44:00.000 Hour: 13 Minutes: 44
Delay (Minutes): -6


Flight Counter: 139
FlightID: 987468473
Carrier: NK
Flight#: 770
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 13:55:00.000 Hour: 13 Minutes: 55
Actual Dep Time: 13:51:00.000 Hour: 13 Minutes: 51
Delay (Minutes): -4


Flight Counter: 140
FlightID: 987495696
Carrier: WN
Flight#: 1479
Airport: MCO
Date: 2019-01-22
Status: L
Scheduled Dep Time: 13:55:00.000 Hour: 13 Minutes: 55
Actual 

In [82]:
# Describe() for Date Range A
print(MCO_20190122_flights_df)


    Counter         ID Carrier Number Airport        Date Status  \
1         1  987468344      NK    432     MCO  2019-01-22      L   
2         2  987425422      B6     52     MCO  2019-01-22      L   
3         3  987442628      F9   1011     MCO  2019-01-22      L   
4         4  987495301      WN   1145     MCO  2019-01-22      L   
5         5  987410386      AA   1110     MCO  2019-01-22      L   
6         6  987412099      AA   2673     MCO  2019-01-22      L   
7         7  987425003      B6   1695     MCO  2019-01-22      L   
8         8  987498353      WN    710     MCO  2019-01-22      D   
9         9  987486421      UA   1863     MCO  2019-01-22      L   
10       10  987718005      5X   5515     MCO  2019-01-22      L   
11       11  987611093      1I    326     MCO  2019-01-22      L   
12       12  987486035      UA   1239     MCO  2019-01-22      L   
13       13  987442641      F9   1178     MCO  2019-01-22      L   
14       14  987485924      UA   1178     MCO  2

In [ ]:
# Describe() for Date Range B
# Compare total number of flights, average flights per day

# Plot average delay by hour in a random day during Date Range A
# Plot average delay by hour in a random day during Date Range B